In [29]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
from scipy.stats import norm,t

In [24]:
def getData(stocks, start, end):
    
    stocksData = yf.download(stocks,start= start,end= end)
    stocksData = stocksData['Close']
    
    pct_return = stocksData.pct_change().dropna()
    
    meanR = pct_return.mean()
    covMatrix = pct_return.cov()
    
    return pct_return,meanR,covMatrix

def portfolio_Performance(weights, meanR,covMatrix,time):
    ret = np.sum(weights*meanR)*time
    std_dev = np.sqrt(np.dot(weights.T , np.dot(covMatrix,weights))) * np.sqrt(time)
    return ret,std_dev


stockList = ['CBA','BHP','AAPL','AMZN','IBM']

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 500)

returns,meanR,covMatrix = getData(stockList,start =startDate,end =endDate)


weights = np.random.random(len(stockList))
weights = weights/np.sum(len(stockList))

returns['Portfolio'] = returns.dot(weights)
print(returns.head())

[*********************100%%**********************]  5 of 5 completed
                AAPL      AMZN       BHP      CBA       IBM  Portfolio
Date                                                                  
2022-06-15  0.020111  0.052390  0.033016  0.00000  0.009873   0.013778
2022-06-16 -0.039651 -0.037243 -0.028087  0.00000 -0.010142  -0.014402
2022-06-17  0.011533  0.024696 -0.040691  0.00000 -0.004791  -0.002576
2022-06-21  0.032761  0.023159 -0.000692  0.00000  0.020960   0.007334
2022-06-22 -0.003827  0.002484 -0.036556  0.37931 -0.005586   0.007449


In [51]:
def historicalVaR(returns,alpha =5):
    VaR = np.percentile(returns['Portfolio'], alpha)
    return VaR

value_at_Risk = historicalVaR(returns,alpha=5)
print(value_at_Risk)

-0.01127956070500292


In [81]:
returns['Portfolio'][belowVaR]


Date
2022-06-15    0.013778
2022-06-16   -0.014402
2022-06-17   -0.002576
2022-06-21    0.007334
2022-06-22    0.007449
                ...   
2023-10-20   -0.009705
2023-10-23    0.001147
2023-10-24    0.008105
2023-10-25   -0.006302
2023-10-26   -0.004529
Name: Portfolio, Length: 344, dtype: float64

In [26]:
def historicalES(returns,alpha =5):
    belowVaR = returns['Portfolio'] < value_at_Risk
    return returns['Portfolio'][belowVaR].mean()

print(historicalES(returns,alpha=5))

-0.014247047060962205


In [27]:
# Lets assume the initial investment of $ 500000 and we are looking at a time horizon of 1 day for VaR and ES.

time = 1
Initial_Investment = 500000

Portfolio_R,Portfolio_Std = portfolio_Performance(weights, meanR,covMatrix,time)
VaR =- historicalVaR(returns,alpha =5)*np.sqrt(time)
ES =- historicalES(returns,alpha =5)*np.sqrt(time)

print("Expected Portfolio Returns:" ,round(Portfolio_R*Initial_Investment,2))
print("Value at Risk at 95th percentile:", round(VaR*Initial_Investment,2))
print("Expected Shortfall at 95th percentile: ", round(ES* Initial_Investment,2))

Expected Portfolio Returns: 178.91
Value at Risk at 95th percentile: 5639.78
Expected Shortfall at 95th percentile:  7123.52


In [38]:
# Parametric VaR and ES

alpha = 5

def Parameteric_VaR(Portfolio_Return,Portfolio_Std,distribution='normal'):
    VaR = Portfolio_Return + Portfolio_Std*norm.ppf(alpha/100)
    return VaR

def Parameteric_ES(Portfolio_Return,Portfolio_Std,distribution='normal'):
    ES = Portfolio_Return + Portfolio_Std*norm.pdf(norm.ppf(alpha/100))/(alpha/100)
    return ES

normal_VaR = -Parameteric_VaR(Portfolio_R,Portfolio_Std)
normal_ES =  Parameteric_ES(Portfolio_R,Portfolio_Std)

print("Parametric Value at Risk at 95th percentile:", round(normal_VaR*Initial_Investment,2))
print("Parametric Expected Shortfall at 95th percentile: ", round(normal_ES* Initial_Investment,2))       

Parametric Value at Risk at 95th percentile: 5922.63
Parametric Expected Shortfall at 95th percentile:  7830.49


In [99]:
# VaR and ES using Monte Carlo Simulation

Portfolio_R,Portfolio_Std = portfolio_Performance(weights, meanR,covMatrix,time)

def random_z_score():
    return np.random.normal(0, 1)

def scenario(Initial_Investment,Portfolio_R,Portfolio_Std,z_score,time):
    return Initial_Investment*Portfolio_R*time + Initial_Investment*Portfolio_Std*z_score*np.sqrt(time)


simulations = 100000
scenario_ret = []

for i in range(simulations):
    z_score=random_z_score()
    scenario_ret.append(scenario(Initial_Investment,Portfolio_R,Portfolio_Std,z_score,time))


    
scenario_ret = pd.Series(scenario_ret)


In [107]:
def MCS_VaR(scenario_ret,alpha =5):
    VaR = np.percentile(scenario_ret, alpha)
    return VaR

value_at_Risk_MCS = MCS_VaR(scenario_ret,alpha=5)
print(-value_at_Risk_MCS)

5970.819850212035


In [108]:
def MCS_ES(scenario_ret,alpha=5):
    belowVaR_MCS = scenario_ret <value_at_Risk_MCS
    return scenario_ret[belowVaR_MCS].mean()

Expected_Shortfall_MCS = MCS_ES(scenario_ret,alpha=5)

print(-Expected_Shortfall_MCS)

7499.913398943202
